In [1]:
from google.colab import drive
import os, json

# Подключаем Диск
drive.mount('/content/drive')

BASE_PATH = '/content/drive/MyDrive/Нафаня бот'
os.makedirs(BASE_PATH, exist_ok=True)

# === Обновляем конфиг с Telegram ===
config_path = os.path.join(BASE_PATH, 'nafani_config.json')
config = {
    "auto_start": True,
    "use_gemini": True,
    "use_binance": True,
    "binance_mode": "sandbox",
    "log_level": "detailed",
    "alert_thresholds": {
        "BTC": 0.01,
        "USDT": 10
    },
    "telegram": {
        "enabled": True,
        "bot_token": "7629814043:AAEK_tA9lDRaMKKOQspcZj1a9URfC6eED60",
        "chat_id": "1379805039"
    }
}

with open(config_path, 'w') as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

# === Файл app.py для Render ===
app_code = """from flask import Flask, request
import requests

app = Flask(__name__)

BOT_TOKEN = "7629814043:AAEK_tA9lDRaMKKOQspcZj1a9URfC6eED60"

# Простейшие команды
def get_status():
    return "Gemini: Да\\nBinance: Да\\nБаланс: BTC 0.05, USDT 15"

def get_balance():
    return "Баланс Binance: BTC 0.05, USDT 15"

def get_logs():
    return "Последние действия:\\n- [INFO] Запуск ядра\\n- [ALERT] Баланс низкий"

def analyze_text(query):
    return f"Анализ Gemini: {query} → Пример ответа."

@app.route("/webhook", methods=["POST"])
def webhook():
    data = request.get_json()
    if "message" in data:
        chat_id = data["message"]["chat"]["id"]
        text = data["message"]["text"]

        if text == "/статус":
            reply = get_status()
        elif text == "/баланс":
            reply = get_balance()
        elif text == "/последние":
            reply = get_logs()
        elif text.startswith("/анализ"):
            query = text.replace("/анализ", "").strip()
            reply = analyze_text(query)
        else:
            reply = "Команды:\\n/статус\\n/баланс\\n/последние\\n/анализ <запрос>"

        requests.post(f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage",
                      data={"chat_id": chat_id, "text": reply})

    return {"ok": True}

@app.route("/")
def home():
    return "Nafanya Bot is running!"
"""

with open(os.path.join(BASE_PATH, 'app.py'), 'w') as f:
    f.write(app_code)

# === requirements.txt ===
requirements = """flask
requests
"""
with open(os.path.join(BASE_PATH, 'requirements.txt'), 'w') as f:
    f.write(requirements)

# === README_DEPLOY.txt ===
readme = """
НАСТРОЙКА TELEGRAM-БОТА НА RENDER

1. Зарегистрируйся на https://render.com/
2. Создай новый Web Service → подключи папку с app.py (или загрузи ZIP).
3. В Settings → добавь переменные окружения:
   BOT_TOKEN = твой токен бота
   (chat_id уже встроен в код, если один пользователь)
4. Укажи:
   - Build Command: pip install -r requirements.txt
   - Start Command: python app.py
5. После деплоя получи URL вида https://имя.onrender.com
6. Установи webhook:
   https://api.telegram.org/bot<ТОКЕН>/setWebhook?url=<URL>/webhook

После этого бот начнет принимать команды напрямую в Telegram.
"""
with open(os.path.join(BASE_PATH, 'README_DEPLOY.txt'), 'w') as f:
    f.write(readme)

print("Файлы для Telegram-бота созданы на Google Диске в папке 'Нафаня бот'.")

Mounted at /content/drive
Файлы для Telegram-бота созданы на Google Диске в папке 'Нафаня бот'.
